# Password Manager Design and Implementation
SecureSphere Innovations is dedicated to providing top-tier protection against digital threats for businesses, financial institutions, and government agencies. One of the critical challenges faced by clients is securely managing a wide range of application and domain passwords.

To address this, SecureSphere Innovations is implementing a Secure Password Generator and Manager. This solution enables users to generate strong, random passwords and manage them along with it logs are also mantained of all the ongoing tasks. With user authentication integrated into the system, authorized personnel can safely add, update, retrieve, or delete passwords, ensuring that sensitive information remains secure and easily accessible when needed. The approach leverages modern security practices to ensure both ease of use and the highest level of protection against cyber threats.

## Password Manager Capability Requirements
- Enable users to generate strong, random passwords
- Manage the passwords generated by users
- Log all the key password transactions
- Enable user authentication and authorization
- Only authorized roles/persons can add, update, retrieve or delete passwords
- Store passwords securely

## Use Cases
#### Add a new password
- User logs in with their password manager (main) password
- User chooses to add a password
- User chooses the password to be either auto-generated by the PM or through manual input
- PM validates the password against a strong password policy
- If acceptable by the policy, the password is encrypted and stored within password manager db, along with associated 'domain'

#### Update a password
- User logs in with their password manager (main) password
- User chooses to update a password
- User provides domain name that the password is stored against
- PM searches for the domain entry and if found, asks user to input updated password.
- User chooses the password to be either auto-generated by the PM or through manual input
- PM validates the password against a strong password policy
- If acceptable by the policy, the password is encrypted and stored within password manager db

#### Retrieve a password
- User logs in with their password manager (main) password
- User chooses to update a password
- User provides domain name that the password is stored against
- PM searches for the key-password entry and if found, decrypts it and outputs on prompt (Note that if UI is to be built for this, the password should not be shown as plaintext on screen. It must be masked and user should be allowed to copy it)

#### Delete a password
- User logs in with their password manager (main) password
- User chooses to update a password
- User provides domain name that the password is stored against
- PM searches for the key-password entry and if found, asks for confirmation to delete
- If user says 'yes', password is removed from the PM database

## Implementation / Low-level Design
#### Functions
- login(user_key, main_password): Logs user into the password manager.
- generate(): password auto-generation that uses strongest randomizer.
- check_policy(policy, incoming_password): validates incoming password against the password policy to check for strength
- encrypt(main_password, incoming_password): encrypt the incoming password using the main_password (symmetric encryption)
- decrypt(main_password, encrypted_password): decrypt the stored password to return to the user
- add(), update(), retrieve(), delete(): password operations requested by the user 

In [5]:
from cryptography.fernet import Fernet
import base64
import hashlib
import random
import string
import os, json
from datetime import datetime

# Function to derive a key from a master password
def derive_key(master_password):
    # Convert master password to bytes
    password_bytes = master_password.encode()
    # Hash it to get a 32-byte key
    key = hashlib.sha256(password_bytes).digest()
    # Encode in base64 for Fernet
    return base64.urlsafe_b64encode(key)

# Encrypt a password
def encrypt_password(password, master_password):
    key = derive_key(master_password)
    f = Fernet(key)
    encrypted = f.encrypt(password.encode())
    return encrypted

# Decrypt a password
def decrypt_password(encrypted_password, master_password):
    key = derive_key(master_password)
    f = Fernet(key)
    decrypted = f.decrypt(encrypted_password)
    return decrypted.decode()

# Example usage
"""
master_password = "my_master_password"
password_to_encrypt = "my_secret_password"

encrypted = encrypt_password(password_to_encrypt, master_password)
print("Encrypted:", encrypted)

decrypted = decrypt_password(encrypted, master_password)
print("Decrypted:", decrypted)
"""

# Generate a random password with a specified password policy
def generate_password(length=12):
    # Ensure at least one character of each type
    upper = random.choice(string.ascii_uppercase)
    lower = random.choice(string.ascii_lowercase)
    digit = random.choice(string.digits)
    special = random.choice("!@#$%^&*()-_=+[]{}|;:,.<>?")

    # Fill the rest of the password randomly
    remaining_length = length - 4
    all_chars = string.ascii_letters + string.digits + "!@#$%^&*()-_=+[]{}|;:,.<>?"
    remaining = ''.join(random.choices(all_chars, k=remaining_length))

    # Combine all characters and shuffle
    password_list = list(upper + lower + digit + special + remaining)
    random.shuffle(password_list)
    password = ''.join(password_list)

    # Check it doesn’t contain "password" or common names
    forbidden = ["password", "admin", "user", "test"]  # add more as needed
    if not any(word.lower() in password.lower() for word in forbidden):
        return password

"""
# Example usage
new_password = generate_password()
print("Generated Password:", new_password)
"""

# Function to check password strength
def is_strong_password(password):
    if len(password) < 12:
        return False
    if not any(c.isupper() for c in password):
        return False
    if not any(c.islower() for c in password):
        return False
    if not any(c.isdigit() for c in password):
        return False
    if not any(c in "!@#$%^&*()-_=+[]{}|;:,.<>?" for c in password):
        return False
    return True

# Example usage
"""
password =  "StrongP@ssw0rd!"
if is_strong_password(password):
    print("Password is strong.")
else:
    print("Password is weak.")
""" 
# Function to validate a password against a policy
def validate_password(password, policy):
    if len(password) < policy.get('min_length', 8):
        return False
    if policy.get('require_uppercase') and not any(c.isupper() for c in password):
        return False
    if policy.get('require_lowercase') and not any(c.islower() for c in password):
        return False
    if policy.get('require_digit') and not any(c.isdigit() for c in password):
        return False
    if policy.get('require_special') and not any(c in "!@#$%^&*()-_=+[]{}|;:,.<>?" for c in password):
        return False
    return True  
# Example usage
"""
password_policy = {
    'min_length': 12,
    'require_uppercase': True,
    'require_lowercase': True,
    'require_digit': True,
    'require_special': True
}
password =  "ValidP@ssw0rd123" 
if validate_password(password, password_policy):
    print("Password meets the policy requirements.")
else:
    print("Password does not meet the policy requirements.")
"""
# Function to add a password entry
def add_password(username, domain, password, master_password):
    encrypted_password = encrypt_password(password, master_password)
    user_data = {
        'username': username,
        'domain': domain,
        'password': str(encrypted_password),  # Store as string for JSON compatibility
        'created_at': datetime.now().isoformat()
    }
    
    # Save to a JSON file
    if not os.path.exists('password_manager.json'):
        with open('password_manager.json', 'w') as f:
            json.dump([], f)
    
    with open('password_manager.json', 'r+') as f:
        data = json.load(f)
        data.append(user_data)
        f.seek(0)
        json.dump(data, f, indent=4)

# Function to retrieve specific password
def get_password(username, domain, master_password): 
    if not os.path.exists('password_manager.json'):
        return []
    
    with open('password_manager.json', 'r') as f:
        data = json.load(f)
    
    decrypted_data = []
    for entry in data:
        if entry['username'] == username and entry['domain'] == domain:
            decrypted_entry = {
                'username': entry['username'],
                'domain': entry['domain'],
                'password': decrypt_password(entry['password'], master_password),
                'created_at': entry['created_at']
            }
            decrypted_data.append(decrypted_entry)
    if not decrypted_data:
        print("No matching entry found.")
    
    return decrypted_data

# Function to update a password entry
def update_password(username, domain, new_password, master_password):
    if not os.path.exists('password_manager.json'):
        print("No password entries found.")
        return
    
    with open('password_manager.json', 'r+') as f:
        data = json.load(f)
        updated = False
        for entry in data:
            if entry['username'] == username and entry['domain'] == domain:
                entry['password'] = str(encrypt_password(new_password, master_password))
                entry['created_at'] = datetime.now().isoformat()
                updated = True
                break
        
        if not updated:
            print("No matching entry found to update.")
            return
        
        f.seek(0)
        json.dump(data, f, indent=4)
# Function to delete a password entry
def delete_password(username, domain):
    if not os.path.exists('password_manager.json'):
        print("No password entries found.")
        return
    
    with open('password_manager.json', 'r+') as f:
        data = json.load(f)
        new_data = [entry for entry in data if not (entry['username'] == username and entry['domain'] == domain)]
        
        if len(new_data) == len(data):
            print("No matching entry found to delete.")
            return
        
        f.seek(0)
        f.truncate()
        json.dump(new_data, f, indent=4)

        # Function to accept user input for password management
def password_manager(): 
    master_password = input("Enter your master password: ")
    
    while True:
        print("\nPassword Manager Menu:")
        print("1. Add Password")
        print("2. Get Password")
        print("3. Update Password")
        print("4. Delete Password")
        print("5. Exit")
        
        choice = input("Choose an option: ")
        
        if choice == '1':
            username = input("Enter username: ")
            domain = input("Enter domain: ")
            choice = input("Do you want to generate a random password? (yes/no): ")
            if choice.lower() == 'yes':
                password = generate_password()
                print(f"Generated Password: {password}")
            else:
                password = input("Enter password: ")
            add_password(username, domain, password, master_password)
            print("Password added successfully.")
        
        elif choice == '2':
            username = input("Enter username: ")
            domain = input("Enter domain: ")
            passwords = get_password(username, domain, master_password)
            if passwords:
                for pwd in passwords:
                    print(f"Username: {pwd['username']}, Domain: {pwd['domain']}, Password: {pwd['password']}, Created At: {pwd['created_at']}")
        
        elif choice == '3':
            username = input("Enter username: ")
            domain = input("Enter domain: ")
            choice = input("Do you want to generate a new random password? (yes/no): ")
            if choice.lower() == 'yes':
                new_password = generate_password()
                print(f"Generated New Password: {new_password}")
            else:
                new_password = input("Enter new password: ")
            update_password(username, domain, new_password, master_password)
            print("Password updated successfully.")
        
        elif choice == '4':
            username = input("Enter username: ")
            domain = input("Enter domain: ")
            delete_password(username, domain)
            print("Password deleted successfully.")
        
        elif choice == '5':
            print("Exiting Password Manager.")
            break
        
        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    password_manager()  
    



Password Manager Menu:
1. Add Password
2. Get Password
3. Update Password
4. Delete Password
5. Exit
Generated Password: 6+Re=j2a7Zb[


JSONDecodeError: Expecting value: line 5 column 21 (char 87)